In [0]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import time
import sklearn
from sklearn.model_selection import train_test_split
from random import shuffle
import seaborn as sns  

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path="/content/drive/My Drive/yelp_final_data/"

### This notebook is for preparing train and test set for Wide and Deep model.

There are two parts:
1. Prepare the large train and test set using the test index we prepared before.
2. Segment the test set into different levels of users and business.

Read the dataset we prepared before. We only want the restaurant that has been rated more than twice and the users that rated at least 5 times.

Read the test index we saved before, so that we split the data into train and test set.

In [4]:
#start_time=time.time()
review=pd.read_csv(path+'review.csv')
del review['text_review']
review['freq_business'] = review.groupby('business_id')['business_id'].transform('count')
review2=review.loc[review['freq_business']>2]
review2['freq_user'] = review2.groupby('user_id')['user_id'].transform('count')
review3=review2.loc[review2['freq_user']>=5]
review3=review3.reset_index()
test_idx=pd.read_csv(path+'all_test_idx_df2.csv')
test_idx=test_idx.rename({'0':'index'},axis=1)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Using the test row index we saved before to get train and test set.

In [0]:
test=review3.loc[review3['index'].isin(test_idx['index'])]
train=review3.loc[~review3['index'].isin(test_idx['index'])]

In [0]:
train.head(3)

,index,user_id,business_id,rating_review,date_review,useful_review,funny_review,cool_review,freq_business,freq_user
0,0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-05-07 04:34:36,6.0,1.0,0.0,183,10
1,1,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,2016-11-09 20:09:03,3.0,0.0,0.0,20,9
2,2,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,3.0,2016-05-07 01:21:02,5.0,4.0,5.0,108,336


Read the user and business dataset and join the table on the user_id and business_id.

In [0]:
user=pd.read_csv(path+'user2.csv')
business=pd.read_csv(path+'business.csv')
business=business.rename(columns={"business_ids": "business_id"})

In [0]:
train = pd.merge(train, user, on='user_id')
test=pd.merge(test, user, on='user_id')

In [0]:
train = pd.merge(train, business, on='business_id')
test = pd.merge(test, business, on='business_id')

In [0]:
train.head(2)

,index,user_id,business_id,rating_review,date_review,useful_review,funny_review,cool_review,freq_business,freq_user,name_x,yelping_since,review_count_x,friends,useful,funny,cool,fans,elite,average_stars,compliment_more,compliment_cute,compliment_funny,num_friends,num_elite,name_y,address,city,state,postal_code,latitude,longtitude,stars,review_count_y,is_open,attributes,categories,hours
0,0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-05-07 04:34:36,6.0,1.0,0.0,183,10,Stefan,2008-07-10 17:24:47,10,frQs7y5qa-X1pvAM0sJe1w,18,4,0,0,NaN,2.0,0,0,0,1,1,MountainView Hospital,3100 N Tenaya Way,Las Vegas,NV,89128,36.215546,-115.248798,2.5,238,1,"{'ByAppointmentOnly': 'False', 'GoodForKids': ...","Fitness & Instruction, Doctors, Health & Medic...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
1,186281,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-03-27 14:17:13,3.0,0.0,0.0,183,10,Stefan,2008-07-10 17:24:47,10,frQs7y5qa-X1pvAM0sJe1w,18,4,0,0,NaN,2.0,0,0,0,1,1,MountainView Hospital,3100 N Tenaya Way,Las Vegas,NV,89128,36.215546,-115.248798,2.5,238,1,"{'ByAppointmentOnly': 'False', 'GoodForKids': ...","Fitness & Instruction, Doctors, Health & Medic...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


Select the features we want. 

In [0]:
train1=train[['index','user_id','business_id','city','state',"average_stars",'useful_review','funny_review','cool_review','compliment_more','compliment_cute',
'num_friends','stars','useful','funny','cool','fans','compliment_funny','categories',"rating_review"]]
test1=test[['index','user_id','business_id','city','state',"average_stars",'useful_review','funny_review','cool_review','compliment_more','compliment_cute',
'num_friends','stars','useful','funny','cool','fans','compliment_funny','categories',"rating_review"]]

In [0]:
train1.head(2)

,index,user_id,business_id,city,state,average_stars,useful_review,funny_review,cool_review,compliment_more,compliment_cute,num_friends,stars,useful,funny,cool,fans,compliment_funny,rating_review
0,0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,Las Vegas,NV,2.0,6.0,1.0,0.0,0,0,1,2.5,18,4,0,0,0,1.0
1,186281,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,Las Vegas,NV,2.0,3.0,0.0,0.0,0,0,1,2.5,18,4,0,0,0,1.0


In [0]:
# user_id_addresses = train1.user_id.unique()
# user_id_dict = dict(zip(user_id_addresses, range(len(user_id_addresses))))
# train1=train1.applymap(lambda s: user_id_dict.get(s) if s in user_id_dict else s)
# test1=test1.applymap(lambda s: user_id_dict.get(s) if s in user_id_dict else s)

In [0]:
# total_business_id=list(train1.business_id.unique())+list(test1.business_id.unique())

In [0]:
# business_id_dict = dict(zip(total_business_id, range(len(total_business_id))))

In [0]:

# train1=train1.applymap(lambda s: business_id_dict.get(s) if s in business_id_dict else s)
# test1=test1.applymap(lambda s: business_id_dict.get(s) if s in business_id_dict else s)

In [0]:
# train1.head(2)

,user_id,business_id,city,state,average_stars,useful_review,funny_review,cool_review,compliment_more,compliment_cute,num_friends,stars,useful,funny,cool,fans,compliment_funny,rating_review
0,0,151026,Las Vegas,NV,2.0,6.0,1.0,0.0,0,0,1,2.5,18,4,0,0,0,1.0
1,0,151026,Las Vegas,NV,2.0,3.0,0.0,0.0,0,0,1,2.5,18,4,0,0,0,1.0


In [0]:
del train1['index']
del test1['index']

Check if there are missing values.

In [10]:
train1.isnull().sum()

index                 0
user_id               0
business_id           0
city                  0
state                 0
average_stars         0
useful_review         2
funny_review          2
cool_review           2
compliment_more       0
compliment_cute       0
num_friends           0
stars                 0
useful                0
funny                 0
cool                  0
fans                  0
compliment_funny      0
categories          554
rating_review         0
dtype: int64

Since there are not many missing values, we fill them with zero, and we fill the missing value in the 'categories' column as 'No category'

In [11]:
train1['useful_review']=train1['useful_review'].fillna(float(0))
train1['funny_review']=train1['funny_review'].fillna(float(0))
train1['cool_review']=train1['cool_review'].fillna(float(0))
train1 = train1.fillna('No_category')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [12]:
train1.isnull().sum()

index               0
user_id             0
business_id         0
city                0
state               0
average_stars       0
useful_review       0
funny_review        0
cool_review         0
compliment_more     0
compliment_cute     0
num_friends         0
stars               0
useful              0
funny               0
cool                0
fans                0
compliment_funny    0
categories          0
rating_review       0
dtype: int64

Done filling missing value for train set.

Check missing value for test set.

In [13]:
test1.isnull().sum()

index                 0
user_id               0
business_id           0
city                  0
state                 0
average_stars         0
useful_review         0
funny_review          0
cool_review           0
compliment_more       0
compliment_cute       0
num_friends           0
stars                 0
useful                0
funny                 0
cool                  0
fans                  0
compliment_funny      0
categories          168
rating_review         0
dtype: int64

In [0]:
test1 = test1.fillna('No_category')

In [15]:
test1.isnull().sum()

index               0
user_id             0
business_id         0
city                0
state               0
average_stars       0
useful_review       0
funny_review        0
cool_review         0
compliment_more     0
compliment_cute     0
num_friends         0
stars               0
useful              0
funny               0
cool                0
fans                0
compliment_funny    0
categories          0
rating_review       0
dtype: int64

Done filling missing value for test set.



In [0]:
train1.to_csv(path+'new_train.csv',index=False)
test1.to_csv(path+'new_test.csv',index=False)

### Prepare segmented test set in user and business dimension

Same data preparation logic applies here, we read the segmented userId and businessID we prepared before to segment the test set into three parts. 
For user, we have:
1. Unpopular user test set
2. Midpopular user test set
3. Popular user test set

For business, we have:
1. Unpopular business test set
2. Midpopular business test set
3. Popular business test set


In [5]:
review=pd.read_csv(path+'review.csv')
del review['text_review']
review['freq_business'] = review.groupby('business_id')['business_id'].transform('count')
review2=review.loc[review['freq_business']>2]
review2['freq_user'] = review2.groupby('user_id')['user_id'].transform('count')
review3=review2.loc[review2['freq_user']>=5]
review3=review3.reset_index()
test_idx=pd.read_csv(path+'all_test_idx_df2.csv')
test_idx=test_idx.rename({'0':'index'},axis=1)
test=review3.loc[review3['index'].isin(test_idx['index'])]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [0]:
path1="/content/drive/My Drive/yelp_segmented_data/"

In [0]:
unpopular_userid=pd.read_csv(path1+'unpopular_user_ID.csv')
midpopular_userid=pd.read_csv(path1+'midpopular_user_ID.csv')
popular_userid=pd.read_csv(path1+'popular_user_ID.csv')

In [0]:
unpopular_user=test.loc[test['user_id'].isin(unpopular_userid['userId'])]
midpopular_user=test.loc[test['user_id'].isin(midpopular_userid['userId'])]
popular_user=test.loc[test['user_id'].isin(popular_userid['userId'])]

In [0]:
user=pd.read_csv(path+'user2.csv')
business=pd.read_csv(path+'business.csv')
business=business.rename(columns={"business_ids": "business_id"})

In [0]:
unpopular_user=pd.merge(unpopular_user, user, on='user_id')
midpopular_user=pd.merge(midpopular_user, user, on='user_id')
popular_user=pd.merge(popular_user, user, on='user_id')

In [0]:
unpopular_user = pd.merge(unpopular_user, business, on='business_id')
midpopular_user = pd.merge(midpopular_user, business, on='business_id')
popular_user = pd.merge(popular_user, business, on='business_id')

In [0]:
unpopular_user=unpopular_user[['index','user_id','business_id','city','state',"average_stars",'useful_review','funny_review','cool_review','compliment_more','compliment_cute',
'num_friends','stars','useful','funny','cool','fans','compliment_funny','categories',"rating_review"]]
midpopular_user=midpopular_user[['index','user_id','business_id','city','state',"average_stars",'useful_review','funny_review','cool_review','compliment_more','compliment_cute',
'num_friends','stars','useful','funny','cool','fans','compliment_funny','categories',"rating_review"]]
popular_user=popular_user[['index','user_id','business_id','city','state',"average_stars",'useful_review','funny_review','cool_review','compliment_more','compliment_cute',
'num_friends','stars','useful','funny','cool','fans','compliment_funny','categories',"rating_review"]]

In [0]:
unpopular_user['useful_review']=unpopular_user['useful_review'].fillna(float(0))
unpopular_user['funny_review']=unpopular_user['funny_review'].fillna(float(0))
unpopular_user['cool_review']=unpopular_user['cool_review'].fillna(float(0))
unpopular_user = unpopular_user.fillna('No_category')

midpopular_user['useful_review']=midpopular_user['useful_review'].fillna(float(0))
midpopular_user['funny_review']=midpopular_user['funny_review'].fillna(float(0))
midpopular_user['cool_review']=midpopular_user['cool_review'].fillna(float(0))
midpopular_user = midpopular_user.fillna('No_category')

popular_user['useful_review']=popular_user['useful_review'].fillna(float(0))
popular_user['funny_review']=popular_user['funny_review'].fillna(float(0))
popular_user['cool_review']=popular_user['cool_review'].fillna(float(0))
popular_user = popular_user.fillna('No_category')


In [0]:
unpopular_user.to_csv(path+'unpopular_user.csv',index=False)
midpopular_user.to_csv(path+'midpopular_user.csv',index=False)
popular_user.to_csv(path+'popular_user.csv',index=False)

In [0]:
unpopular_businessid=pd.read_csv(path1+'unpopular_business_ID.csv')
midpopular_businessid=pd.read_csv(path1+'midpopular_business_ID.csv')
popular_businessid=pd.read_csv(path1+'popular_business_ID.csv')

In [0]:
unpopular_business=test.loc[test['business_id'].isin(unpopular_businessid['businessId'])]
midpopular_business=test.loc[test['business_id'].isin(midpopular_businessid['businessId'])]
popular_business=test.loc[test['business_id'].isin(popular_businessid['businessId'])]

In [0]:
unpopular_business=pd.merge(unpopular_business, user, on='user_id')
midpopular_business=pd.merge(midpopular_business, user, on='user_id')
popular_business=pd.merge(popular_business, user, on='user_id')

In [0]:
unpopular_business = pd.merge(unpopular_business, business, on='business_id')
midpopular_business = pd.merge(midpopular_business, business, on='business_id')
popular_business= pd.merge(popular_business, business, on='business_id')

In [0]:
unpopular_business=unpopular_business[['index','user_id','business_id','city','state',"average_stars",'useful_review','funny_review','cool_review','compliment_more','compliment_cute',
'num_friends','stars','useful','funny','cool','fans','compliment_funny','categories',"rating_review"]]
midpopular_business=midpopular_business[['index','user_id','business_id','city','state',"average_stars",'useful_review','funny_review','cool_review','compliment_more','compliment_cute',
'num_friends','stars','useful','funny','cool','fans','compliment_funny','categories',"rating_review"]]
popular_business=popular_business[['index','user_id','business_id','city','state',"average_stars",'useful_review','funny_review','cool_review','compliment_more','compliment_cute',
'num_friends','stars','useful','funny','cool','fans','compliment_funny','categories',"rating_review"]]

In [0]:
unpopular_business['useful_review']=unpopular_business['useful_review'].fillna(float(0))
unpopular_business['funny_review']=unpopular_business['funny_review'].fillna(float(0))
unpopular_business['cool_review']=unpopular_business['cool_review'].fillna(float(0))
unpopular_business = unpopular_business.fillna('No_category')

midpopular_business['useful_review']=midpopular_business['useful_review'].fillna(float(0))
midpopular_business['funny_review']=midpopular_business['funny_review'].fillna(float(0))
midpopular_business['cool_review']=midpopular_business['cool_review'].fillna(float(0))
midpopular_business = midpopular_business.fillna('No_category')

popular_business['useful_review']=popular_business['useful_review'].fillna(float(0))
popular_business['funny_review']=popular_business['funny_review'].fillna(float(0))
popular_business['cool_review']=popular_business['cool_review'].fillna(float(0))
popular_business = popular_business.fillna('No_category')


In [0]:
unpopular_business.to_csv(path+'unpopular_business.csv',index=False)
midpopular_business.to_csv(path+'midpopular_business.csv',index=False)
popular_business.to_csv(path+'popular_business.csv',index=False)